# Sesión 11 A

> **Objetivos:**
> - Entender los diferentes tipos de consultas que le podemos hacer a un modelo gráfico.

### 1. Introducción

En la sesión pasada comenzamos a estudiar el tema de _redes bayesianas_ y vimos que:

En una red bayesiana, los _tipos de razonamiento_ describen cómo fluye la información a través del grafo:

| Tipo de razonamiento | Dirección | Ejemplo | Intuición |
|----------------------|------------|----------|-----------|
| **Causal (predictivo)** | Causa → Efecto | $P(\text{Fiebre} \mid \text{Gripe})$ | Predecir consecuencias de causas. |
| **Evidencial (diagnóstico)** | Efecto → Causa | $P(\text{Gripe} \mid \text{Fiebre})$ | Diagnosticar causas a partir de efectos. |
| **Intercausal** | Entre causas de un mismo efecto | $P(\text{Aspersores} \mid \text{Césped mojado}, \text{Lluvia})$ | Explicar relaciones entre causas que comparten un efecto. |

En esta sesión veremos los **tipos de consultas**. Éstos nos dicen _cómo queremos responder_ una pregunta probabilística usando un modelo gráfico.

| Tipo de consulta | Qué calcula | Respuesta |
|------------------|--------------|-----------|
| **Probabilística (posterior query)** | $P(Y \mid \text{Evidencia})$ | Distribución completa sobre $Y$ |
| **MAP (decisión puntual)** | $\arg\max_Y P(Y \mid \text{Evidencia})$ | El valor más probable de $Y$ |

#### 1.1. Consultas probabilísticas

Las **consultas probabilísticas** son el tipo de consulta más común en los modelos gráficos probabilísticos.  

Básicamente, nos permiten responder preguntas del tipo:

> "¿Cuál es la probabilidad de que ocurra algo _(una variable de interés)_?"

> "¿Cuál es la probabilidad de que ocurra algo _(una variable de interés)_, sabiendo que ya observamos otra cosa _(una evidencia)_?"

Por ejemplo:  
> “¿Cuál es la probabilidad de que un estudiante apruebe?”  
> “¿Cuál es la probabilidad de que llueva, dado que el cielo está nublado?”

---

Sea el conjunto completo de variables aleatorias del modelo:

$$
P(X_1, \ldots, X_n)
$$

(modelada a través de una red Bayesiana).

Definimos:

* Un conjunto de **variables de interés**:

  $$
  \bar{Y} \subseteq \{X_1, \dots, X_n\}
  $$

> Son las variables sobre las que queremos conocer.

* Un conjunto de **variables de evidencia (observadas)**:

  $$
  \bar{E} = \bar{e}, \quad \text{con } \bar{E} \subseteq \{X_1, \dots, X_n\}
  $$

> Son las variables cuyo valor ya conocemos (por ejemplo, síntomas observados, resultados medidos, etc.).

* El conjunto restante de variables no observadas ni consultadas:
  $$
  \bar{W} = \{X_1, \dots, X_n\} \setminus (\bar{Y} \cup \bar{E})
  $$

> Son las variables “ocultas” o que no intervienen directamente en la consulta.

El objetivo de la inferencia es **calcular la probabilidad condicional** de las variables de interés dado el conjunto de evidencias observadas:

$$
P(\bar{Y} \mid \bar{E} = \bar{e})
$$

**¿Cómo?**

Por la definición de probabilidad condicional:

$$
\boxed{
P(\bar{Y} \mid \bar{E} = \bar{e}) = \frac{P(\bar{Y}, \bar{e})}{P(\bar{e})}
}
$$

En esta expresión:

- $P(\bar{Y}, \bar{e}) = \sum_{\bar{W}} P(\bar{Y}, \bar{e}, \bar{W})$  

  Recordemos que $\{X_1, \dots, X_n\} = \bar{Y} \cup \bar{E} \cup \bar{W}$.  
  Por lo tanto, los términos dentro de la suma representan las **probabilidades conjuntas** de todas las variables del modelo.

- $P(\bar{e}) = \sum_{\bar{Y}} P(\bar{Y}, \bar{e})$  

  Esta cantidad actúa como una **constante de normalización**, necesaria para convertir la distribución conjunta $P(\bar{Y}, \bar{e})$ en una distribución condicional válida $P(\bar{Y} \mid \bar{e})$.


> Lo anterior es, en esencia, lo que ya habíamos estado realizando con el ejemplo del **modelo del estudiante**.
>
> Recordemos que aplicábamos la **fórmula de la probabilidad condicional** y luego **marginalizábamos** sobre las variables no observadas para obtener las distribuciones condicionales de interés.

In [1]:
from pgmpy.models import BayesianNetwork, DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
import os
from functools import reduce
import operator

In [3]:
import pickle

ruta_modelo = os.path.join('..', 'data', 'student-model.pkl')
with open(ruta_modelo, 'rb') as f:
    student_model = pickle.load(f)

print(type(student_model))

<class 'pgmpy.models.DiscreteBayesianNetwork.DiscreteBayesianNetwork'>


In [4]:
factores = [cpd.to_factor() for cpd in student_model.get_cpds()]
p_joint = reduce(lambda x, y: x * y, factores)

Retomando una de las consultas que hicimos la sesión pasada:

> ¿Cuál es la probabilidad de que un estudiante obtenga una carta de recomendación (R), dado que no estudió mucho (I=0)?

$$P(r^1 | i^0) = \frac{P(r^1, i^0)}{P(i^0)} = \frac{\sum_{D,C,E} P(D, i^0, C, E, r^1)}{\sum_{D,C,E,R} P(D, i^0, C, E, R)} \approx ?$$

In [5]:
p_RI = p_joint.marginalize(variables=['D', 'C', 'E'], inplace=False)
p_I = p_joint.marginalize(variables=['D', 'C', 'E', 'R'], inplace=False)

In [6]:
# numerador
print(p_RI)

+------+------+------------+
| R    | I    |   phi(R,I) |
+======+======+============+
| R(0) | I(0) |     0.4280 |
+------+------+------------+
| R(0) | I(1) |     0.0697 |
+------+------+------------+
| R(1) | I(0) |     0.2720 |
+------+------+------------+
| R(1) | I(1) |     0.2303 |
+------+------+------------+


In [7]:
# denominador
print(p_I)

+------+----------+
| I    |   phi(I) |
+======+==========+
| I(0) |   0.7000 |
+------+----------+
| I(1) |   0.3000 |
+------+----------+


In [8]:
p_RI0 = p_RI.reduce([('I', 0)], inplace=False) # nos quedamos con I=0 (evidencia)
print(p_RI0)

+------+----------+
| R    |   phi(R) |
+======+==========+
| R(0) |   0.4280 |
+------+----------+
| R(1) |   0.2720 |
+------+----------+


In [9]:
pi0 = p_I.get_value(I=0) # normalizador (evidencia)
pi0

np.float64(0.7)

In [10]:
print('R(0) =', p_RI0.values[0]/pi0)
print('R(1) =', p_RI0.values[1]/pi0)

R(0) = 0.6113999999999999
R(1) = 0.38860000000000006


In [11]:
p_R_given_I0_vals = p_RI0.values / pi0  # vector 
print(p_R_given_I0_vals)

[0.6114 0.3886]


La consulta actual:

$$
P(r^1 \mid i^0) = \frac{P(r^1, i^0)}{P(i^0)}
$$

produce una **distribución condicional** sobre $R$, dado que $I = i^0$.  

Obtuvimos:

$$
P(R \mid I = i^0) = [0.6114,\; 0.3886]
$$

que representa:

$$
P(R = r^0 \mid I = i^0) = 0.6114 \quad \text{y} \quad P(R = r^1 \mid I = i^0) = 0.3886
$$

### 1.2. Inferencia Máximo a Posteriori (MAP)

En la sección anterior vimos que la inferencia mediante **probabilidad condicional** nos entrega una **distribución completa** sobre las posibles combinaciones de las variables de interés, dadas las evidencias observadas.

Por ejemplo, al calcular:

$$
P(\bar{Y} \mid \bar{E} = \bar{e})
$$

obtenemos **todas** las probabilidades posibles de $\bar{Y}$ (una tabla que suma 1).  

Sin embargo, a veces **no necesitamos toda la distribución**, sino **el valor más probable**: la combinación de estados de las variables de interés que **maximiza** esa probabilidad condicional.

A esto lo llamamos **Inferencia máximo a posteriori (MAP)**:

$$
MAP(\bar{Y} \mid \bar{E} = \bar{e}) = 
\arg\max_{\bar{y}} P(\bar{Y} = \bar{y} \mid \bar{E} = \bar{e})
$$

El método ``map_query`` nos ayuda a realizar este tipo de consultas en una red bayesiana.

$$P(R \mid i^0)$$

In [12]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(student_model)
r_map = infer.map_query(variables=["R"], #variables = var de interés;
                        evidence={"I": 0})  # evidence = evidencia
print(r_map)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'R': 0}


Este `R:0` nos indica el valor más probable de la variable `R`, dado que `I=0`.

> Traducido: **el valor más probable de `R` es `0` (no obtiene carta de recomendación) cuando `I=0` (estudiante no muy destacado).**

El **máximo a posteriori (MAP)** no siempre coincide con el **máximo sobre las distribuciones marginales**.

![](../images/sesion11-map2.png)

**Figura 1:** Diferencia entre el valor MAP y el valor máximo de las distribuciones marginales.

> En la **Figura 1**, el par más probable conjuntamente es $(A=a^0, B=b^1)$ con $P=0.36$ (MAP conjunto).
>
> Pero si tomamos los valores más probables de cada variable por separado —$A=a^1$ y $B=b^1$ según sus marginales— obtenemos $(a^1, b^1)$, cuya probabilidad conjunta es menor ($0.30$).
>
>maximizar las marginales busca los valores más probables individualmente, mientras que el MAP busca la combinación más probable en conjunto.

### 1.3. Escalando a redes bayesianas grandes

![](../images/sesion9-student-model-factors.png)

**Figura 2:** Factorización del modelo del estudiante.

> Como se mencionó anteriormente, el modelo del estudiante es un ejemplo de *“juguete”*, es decir, un caso simplificado con pocas variables y cardinalidades reducidas.

En modelos reales, las redes bayesianas pueden tener **decenas o cientos de variables** conectadas (como la de la **Figura 3**).

Cada nodo representa una variable aleatoria y las conexiones codifican relaciones de dependencia probabilística.  

El cálculo de una probabilidad como:

$$
P(X \mid \text{evidencia})
$$

implica **sumar o marginalizar** sobre *todas* las demás variables no observadas del modelo:

$$
P(X \mid E=e) = \frac{\sum_Y P(X, Y, E=e)}{P(E=e)}
$$

A medida que crece el número de nodos, esta suma se vuelve **exponencialmente costosa**, porque el número de combinaciones posibles de estados explota.


Veamos lo siguiente:

![Ejemplo](../images/sesion11-ejem.png)

**Figura 3. Modelo diagnóstico de endometriosis**

Piensa en el crecimiento de una red bayesiana:

- Cada variable puede tener varios **estados** (por ejemplo, *sí/no* o *bajo/medio/alto*).
- Cada nodo depende de sus **padres** y sus tablas de probabilidad condicional crecen **exponencialmente** con el número de padres.
- Para calcular una probabilidad marginal o condicional, hay que **sumar o multiplicar sobre todas las combinaciones posibles** de los valores de las variables no observadas.

![](../images/sesion11-endom-full.jpg)

**Figura 4. Modelo diagnóstico de endometriosis completo**

Cuando la red contiene muchas variables (como en el ejemplo de la red diagnóstica de endometriosis u otras más densas), el número de combinaciones posibles se vuelve **gigante** y el tiempo de cómputo crece **más rápido que cualquier función polinomial** del número de nodos.


> Por este motivo, en esta sesión exploraremos **métodos prácticos de inferencia en redes bayesianas**, que permiten realizar los mismos cálculos de forma más eficiente y escalable.

![](../images/locomotive.png)

**Figura 5.** Red bayesiana que modela diversos problemas encontrados en el diagnóstico de locomotoras diésel, sus posibles causas, síntomas y resultados de las pruebas. Retomada de: [BayesFusion](https://www.bayesfusion.com/bayesian-networks/)

```{admonition} Ejemplo de red bayesiana grande
:class: tip

En la **Figura 5** se muestra un ejemplo de una red bayesiana que modela *2,127 variables binarias*.

Si quisiéramos representar la distribución conjunta completa **sin aprovechar ninguna independencia condicional**, sería necesario almacenar

$$
2^{2127} \approx 10^{640}
$$

entradas en una tabla de probabilidad conjunta, una cantidad completamente inabordable.

Gracias a la estructura de dependencias codificada por la red bayesiana, esta distribución puede representarse con

$$6,433 \text{ parámetros independientes}$$

Sin embargo, *reducir la cantidad de parámetros* no implica que la inferencia sea sencilla.

Calcular probabilidades **marginales** o **condicionales** aún puede requerir recorrer una cantidad exponencial de combinaciones posibles de valores, lo que vuelve el proceso computacionalmente intractable en redes grandes.

Esta limitación da lugar a dos enfoques principales para la inferencia en redes bayesianas...
```

#### 1.4. Inferencia exacta vs. inferencia aproximada

Existen dos enfoques principales para hacer inferencia en redes grandes:

1. **Inferencia exacta:**  
   Calcula el valor exacto de las probabilidades, pero puede ser muy costosa computacionalmente.  
   - El algoritmo más común es la **Eliminación de Variables** (*Variable Elimination*).

2. **Inferencia aproximada:**  
   - Utiliza muestreo o métodos numéricos para **estimar** las probabilidades cuando la exacta es inviable.  

[1] R. Collins and N. Fenton, "Bayesian network modelling for early diagnosis and prediction of Endometriosis," *medRxiv*, preprint, Nov. 2020. Disponible: https://www.medrxiv.org/content/10.1101/2020.11.04.20225946v1
